# 3D drone-pendulum system lagrange

In [3]:
import json
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg
import scipy
from scipy import linalg
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
import sympy as sym
from sympy.physics.mechanics import *

In [4]:
# components of position (meters)
o_x, o_y, o_z = sym.symbols('o_x, o_y, o_z')

# yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# states of the pendulum
alpha, alpha_dot = sym.symbols('alpha, alpha_dot')

In [5]:
# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')

In [6]:
m, J_x, J_y, J_z, g, m_pen, l_pen = sym.symbols('m, J_x, J_y, J_z, g, m_pen, l_pen')

In [7]:
o_x, o_y, o_z, alpha, psi, phi, theta = dynamicsymbols(r'o_x o_z o_z \alpha \psi \phi \theta')
o_xd, o_yd, o_zd, alphad, psid, phid, thetad = dynamicsymbols(r'o_x o_z o_z \alpha \psi \phi \theta', 1)
o_xdd, o_ydd, o_zdd, alphadd, psidd, phidd, thetadd = dynamicsymbols(r'o_x o_z o_z \alpha \psi \phi \theta', 2)
t = sym.Symbol('t')

In [8]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

In [9]:
R_1in0 = Rz * Ry * Rx

In [10]:
R_1in0

Matrix([
[cos(\psi(t))*cos(\theta(t)), sin(\phi(t))*sin(\theta(t))*cos(\psi(t)) - sin(\psi(t))*cos(\phi(t)),  sin(\phi(t))*sin(\psi(t)) + sin(\theta(t))*cos(\phi(t))*cos(\psi(t))],
[sin(\psi(t))*cos(\theta(t)), sin(\phi(t))*sin(\psi(t))*sin(\theta(t)) + cos(\phi(t))*cos(\psi(t)), -sin(\phi(t))*cos(\psi(t)) + sin(\psi(t))*sin(\theta(t))*cos(\phi(t))],
[            -sin(\theta(t)),                                          sin(\phi(t))*cos(\theta(t)),                                           cos(\phi(t))*cos(\theta(t))]])

In [11]:
s_pen_1 = sym.Matrix([[l_pen*sym.sin(alpha)], [0.], [l_pen*sym.cos(alpha)]])
s_pen_0 = R_1in0 @ s_pen_1
x_pen_0 = s_pen_0[0]
y_pen_0 = s_pen_0[1]
z_pen_0 = s_pen_0[2]
x_pen_0_dot = sym.simplify(sym.diff(x_pen_0, t))
y_pen_0_dot = sym.simplify(sym.diff(y_pen_0, t))
z_pen_0_dot = sym.simplify(sym.diff(z_pen_0, t))

In [12]:
L = 1/2*m*(o_xd**2 + o_yd**2 + o_zd**2) + 1/2*J_y*(thetad**2) + 1/2*J_x*(phid**2) + 1/2*J_z*(psid**2) + 1/2*m_pen*((o_xd + x_pen_0_dot)**2 + (o_yd + y_pen_0_dot)**2 + (o_zd + z_pen_0_dot)**2) - m*g*o_z - m_pen*g*(o_z + z_pen_0)


0.5*J_x*Derivative(\phi(t), t)**2 + 0.5*J_y*Derivative(\theta(t), t)**2 + 0.5*J_z*Derivative(\psi(t), t)**2 - g*m*o_z(t) - g*m_pen*(-l_pen*sin(\alpha(t))*sin(\theta(t)) + l_pen*cos(\alpha(t))*cos(\phi(t))*cos(\theta(t)) + o_z(t)) + 0.5*m*(Derivative(o_x(t), t)**2 + 2*Derivative(o_z(t), t)**2) + 0.5*m_pen*((l_pen*(-(sin(\phi(t))*sin(\psi(t)) + sin(\theta(t))*cos(\phi(t))*cos(\psi(t)))*sin(\alpha(t))*Derivative(\alpha(t), t) + (-sin(\phi(t))*sin(\theta(t))*cos(\psi(t))*Derivative(\phi(t), t) + sin(\phi(t))*cos(\psi(t))*Derivative(\psi(t), t) - sin(\psi(t))*sin(\theta(t))*cos(\phi(t))*Derivative(\psi(t), t) + sin(\psi(t))*cos(\phi(t))*Derivative(\phi(t), t) + cos(\phi(t))*cos(\psi(t))*cos(\theta(t))*Derivative(\theta(t), t))*cos(\alpha(t)) - sin(\alpha(t))*sin(\psi(t))*cos(\theta(t))*Derivative(\psi(t), t) - sin(\alpha(t))*sin(\theta(t))*cos(\psi(t))*Derivative(\theta(t), t) + cos(\alpha(t))*cos(\psi(t))*cos(\theta(t))*Derivative(\alpha(t), t)) + Derivative(o_x(t), t))**2 + (l_pen*((sin(\

KeyboardInterrupt: 

In [17]:
f_z_0 = R_1in0 @ sym.Matrix([0, 0, f_z])

Equations = sym.Matrix([[sym.diff(sym.diff(L, o_xd), t) - sym.diff(L, o_x) - f_z_0[0]],
             [sym.diff(sym.diff(L, o_yd), t) - sym.diff(L, o_y) - f_z_0[1]],
             [sym.diff(sym.diff(L, o_zd), t) - sym.diff(L, o_z) - f_z_0[2]],
             [sym.diff(sym.diff(L, psid), t) - sym.diff(L, psi) - tau_z],
             [sym.diff(sym.diff(L, thetad), t) - sym.diff(L, theta) - tau_y],
             [sym.diff(sym.diff(L, phid), t) - sym.diff(L, phi) - tau_x],
             [sym.diff(sym.diff(L, alphad), t) - sym.diff(L, alpha)]])

In [18]:
Equations

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [27]:
sym.solve(Equations, [o_xdd, o_ydd, o_zdd, psidd, thetadd, phidd, alphadd])

KeyboardInterrupt: 

In [22]:
ox_ddot_solve = sym.solve(Equations[0], o_xdd)[0]


In [24]:
oy_ddot_solve = sym.solve(Equations[1], o_ydd)[0]

In [26]:
oz

(f_z*sin(\phi(t))*sin(\psi(t)) + f_z*sin(\theta(t))*cos(\phi(t))*cos(\psi(t)) + l_pen*m_pen*sin(\alpha(t))*sin(\phi(t))*sin(\psi(t))*Derivative(\alpha(t), (t, 2)) - 2.0*l_pen*m_pen*sin(\alpha(t))*sin(\phi(t))*sin(\theta(t))*cos(\psi(t))*Derivative(\alpha(t), t)*Derivative(\phi(t), t) + 2.0*l_pen*m_pen*sin(\alpha(t))*sin(\phi(t))*cos(\psi(t))*Derivative(\alpha(t), t)*Derivative(\psi(t), t) - 2.0*l_pen*m_pen*sin(\alpha(t))*sin(\psi(t))*sin(\theta(t))*cos(\phi(t))*Derivative(\alpha(t), t)*Derivative(\psi(t), t) - 2.0*l_pen*m_pen*sin(\alpha(t))*sin(\psi(t))*sin(\theta(t))*Derivative(\psi(t), t)*Derivative(\theta(t), t) + 2.0*l_pen*m_pen*sin(\alpha(t))*sin(\psi(t))*cos(\phi(t))*Derivative(\alpha(t), t)*Derivative(\phi(t), t) + l_pen*m_pen*sin(\alpha(t))*sin(\psi(t))*cos(\theta(t))*Derivative(\psi(t), (t, 2)) + l_pen*m_pen*sin(\alpha(t))*sin(\theta(t))*cos(\phi(t))*cos(\psi(t))*Derivative(\alpha(t), (t, 2)) + l_pen*m_pen*sin(\alpha(t))*sin(\theta(t))*cos(\psi(t))*Derivative(\theta(t), (t, 2)